In [4]:
# Script to convert yolo annotations to voc format

# Sample format
# <annotation>
#     <folder>data</folder>
#     <filename>example.jpg</filename>
#     <path>C:\Users\DiPad\Desktop\try\data\example.jpg</path>
#     <size>
#         <width>1200</width>
#         <height>800</height>
#         <depth>3</depth>
#     </size>
#     <segmented>0</segmented>
#     <object>
#         <name>head</name>
#         <pose>Unspecified</pose>
#         <truncated>0</truncated>
#         <difficult>0</difficult>
#         <bndbox>
#             <xmin>549</xmin>
#             <ymin>251</ymin>
#             <xmax>625</xmax>
#             <ymax>335</ymax>
#         </bndbox>
#     </object>
# <annotation>

import os
# import xml.etree.ElementTree as ET
from lxml import etree as ET
from PIL import Image
from matplotlib.pyplot import imshow
import numpy as np

want = "train"

ANNOTATIONS_DIR_PREFIX = want

DESTINATION_DIR = want

CLASS_MAPPING = {
    '0':'dolphin',
    '1':'fish',
    '2':'trutle',
    '3':'jellyfish',
    '4':'starfish',
    '5':'swordfish'
    # Add classes if needed.
}


def create_root(file_prefix, width, height):
    root = ET.Element("annotation")
    ############################################
    # make sure this is the folder and path for images
    ET.SubElement(root, "folder").text = "val"
    ET.SubElement(root, "filename").text = "{}.jpg".format(file_prefix)
    x = os.getcwd()
    ET.SubElement(root,"path").text = x+"/"+want+"/{}.jpg".format(file_prefix)
    ############################################
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    return root


def create_object_annotation(root, voc_labels):
    for voc_label in voc_labels:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = voc_label[0]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(voc_label[1])
        ET.SubElement(bbox, "ymin").text = str(voc_label[2])
        ET.SubElement(bbox, "xmax").text = str(voc_label[3])
        ET.SubElement(bbox, "ymax").text = str(voc_label[4])
    return root


def create_file(file_prefix, width, height, voc_labels):
    root = create_root(file_prefix, width, height)
    root = create_object_annotation(root, voc_labels)
    tree = ET.ElementTree(root)
    tree.write("{}/{}.xml".format(DESTINATION_DIR, file_prefix),pretty_print=True)


def read_file(file_path):
    print(file_path)
    file_prefix = file_path.split(".txt")[0]
#     print(file_prefix)
    image_file_name = "{}.jpg".format(file_prefix)
#     print(image_file_name)
    img = Image.open("{}/{}".format(want, image_file_name))
#     imshow(np.asarray(img))
    w, h = img.size
#     print(w,h)
    with open(want+'/'+file_path, 'r') as file:
        lines = file.readlines()
        voc_labels = []
        for line in lines:
            voc = []
            line = line.strip()
            data = line.split()
            voc.append(CLASS_MAPPING.get(data[0]))
            bbox_width = float(data[3]) * w
            bbox_height = float(data[4]) * h
            center_x = float(data[1]) * w
            center_y = float(data[2]) * h
            voc.append(round(center_x - (bbox_width / 2)))
            voc.append(round(center_y - (bbox_height / 2)))
            voc.append(round(center_x + (bbox_width / 2)))
            voc.append(round(center_y + (bbox_height / 2)))
            voc_labels.append(voc)
        create_file(file_prefix, w, h, voc_labels)
    print("Processing complete for file: {}".format(file_path))


def start(dir_name):
    if not os.path.exists(DESTINATION_DIR):
        os.makedirs(DESTINATION_DIR)
    for filename in os.listdir(ANNOTATIONS_DIR_PREFIX):
        if filename.endswith('txt'):
            read_file(filename)
        else:
            print("Skipping file: {}".format(filename))


if __name__ == "__main__":
    start(want)

Skipping file: dolphin_53.xml
Skipping file: starfish_108.jpg
jellyfish_45.txt
Processing complete for file: jellyfish_45.txt
Skipping file: starfish_55.jpg
Skipping file: dolphin_2.xml
Skipping file: fish_12.jpg
Skipping file: starfish_74.jpg
Skipping file: fish_119.xml
Skipping file: starfish_118.jpg
Skipping file: turtle_25.xml
Skipping file: dolphin_12.jpg
Skipping file: dolphin_14.jpg
Skipping file: starfish_80.jpg
Skipping file: fish_19.jpg
Skipping file: fish_21.xml
Skipping file: dolphin_92.xml
Skipping file: jellyfish_124.jpg
Skipping file: swordfish_1.xml
jellyfish_123.txt
Processing complete for file: jellyfish_123.txt
Skipping file: dolphin_23.xml
jellyfish_27.txt
Processing complete for file: jellyfish_27.txt
Skipping file: swordfish_0.xml
Skipping file: fish_6.xml
Skipping file: jellyfish_123.jpg
Skipping file: turtle_105.xml
Skipping file: jellyfish_32.jpg
Skipping file: fish_43.xml
Skipping file: turtle_106.jpg
Skipping file: swordfish_56.jpg
Skipping file: fish_59.jpg


In [1]:
import os
os.getcwd()

'/home/dgu/faster-rcnn.pytorch/data'

In [ ]:
# CUDA_VISIBLE_DEVICES=0 python trainval_net.py --dataset pascal_voc --net res101 --bs 4 --lr 4e-3 --lr_decay_step 8 --cuda

In [ ]:
# python demo.py --net res101 --checksession 1 --checkepoch 20 --checkpoint 139 --cuda --load_dir models